# FrostAura Plutus
### Mark 10
This iteration of the decision engine will consist of neural network-based architecture. A Deep Q-learning agent.

## Features
The environment's observable space will consist of features the shape of multiple dimensions as follows.
- Prices of the pair with time.
- A few indicators at various configurations for the closing price.
- Whether we currently have an open stake in this pair or not. 

In [6]:
# Load features from pickle generated by the ./parse_market_data notebook.
import pickle

model_file_path = './data/featurized_market_data.p'

with open(model_file_path, 'rb') as fp:
    featurized_market_data = pickle.load(fp)

In [48]:
# Configure the pair we will work with for testing.
pair_df = featurized_market_data['AAVE_BTC']

In [53]:
# Add non-indicator observation values.
pair_df['staked'] = True
pair_df['comission'] = 0.03

### Dimension 1
The 1st dimension's features consist of 38 items. This dimension is meant to represent all values observable in a flat array per-timestep. 
- The usual open, close, lower, upper and volume.
- The deconstructed time (for seasonality).
- The moving average of the closing price.
- ADX momentum indicator on the close price.
- The RSI on the close price with a period of 14.
- Stoch slow on the close price with a period of 14.
- Bolinger Bands with std 2 on the close price with a period of 14.
- Bolinger Bands with std 3 on the close price with a period of 14.
- Bolinger Bands with std 4 on the close price with a period of 14.
- The RSI on the close price with a period of 20.
- Stoch slow on the close price with a period of 20.
- Bolinger Bands with std 2 on the close price with a period of 20.
- Bolinger Bands with std 3 on the close price with a period of 20.
- Bolinger Bands with std 4 on the close price with a period of 20.
- Whether we have a current stake in this pair. (This means support for one transaction p/pair at a time)
- The trading platform's comission percentage. 0 > % < 1.

In [54]:
pair_df.columns

Index(['time', 'open', 'high', 'low', 'close', 'volume', 'hour', 'day', 'week',
       'month', 'sma', 'rsi_tp14', 'adx', 'stoch_slow_k_tp_14',
       'stoch_slow_d_tp_14', 'bb_upper_tp14_sd2', 'bb_middle_tp14_sd2',
       'bb_lower_tp14_sd2', 'bb_upper_tp14_sd3', 'bb_middle_tp14_sd3',
       'bb_lower_tp14_sd3', 'bb_upper_tp14_sd4', 'bb_middle_tp14_sd4',
       'bb_lower_tp14_sd4', 'rsi_tp20', 'stoch_slow_k_tp_20',
       'stoch_slow_d_tp_20', 'bb_upper_tp20_sd2', 'bb_middle_tp20_sd2',
       'bb_lower_tp20_sd2', 'bb_upper_tp20_sd3', 'bb_middle_tp20_sd3',
       'bb_lower_tp20_sd3', 'bb_upper_tp20_sd4', 'bb_middle_tp20_sd4',
       'bb_lower_tp20_sd4', 'staked', 'comission'],
      dtype='object')

### Dimension 2
The 2nd dimension's features represent timesteps for dimension 1.
- Window length of historical record counts to make observable.
#### Example
In the example below we choose to have a window of 4 historical adjacent to an inclusive current record.
At this point, dimension 1 (38) + dimension 2 (5) = ***double[5][36]***
| | | | | |
|---|---|---|---|---|
| 0 | 1 | 2 | .. | 38 |
| 1 | - | - | - | - |
| 2 | - | - | - | - |
| .. | - | - | - | - |
| 5 | - | - | - | - |

In [47]:
pair_to_trade = featurized_market_data['AAVE_BTC']
pair_to_trade[:5]

,time,open,high,low,close,volume,hour,day,week,month,...,stoch_slow_d_tp_20,bb_upper_tp20_sd2,bb_middle_tp20_sd2,bb_lower_tp20_sd2,bb_upper_tp20_sd3,bb_middle_tp20_sd3,bb_lower_tp20_sd3,bb_upper_tp20_sd4,bb_middle_tp20_sd4,bb_lower_tp20_sd4
0,2020-10-15 03:00:00,0.004512,0.005087,0.004081,0.004126,10209.72,3,15,42,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-10-15 04:00:00,0.004136,0.004192,0.004095,0.004167,2670.00,4,15,42,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-10-15 05:00:00,0.004161,0.004368,0.004161,0.004296,2766.22,5,15,42,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-10-15 06:00:00,0.004298,0.004331,0.004270,0.004303,1523.50,6,15,42,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-10-15 07:00:00,0.004306,0.004333,0.004260,0.004300,1048.25,7,15,42,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Next
### Mark 11
Add another dimension to the observable space to have all pairs available as to allow the agent to learn causal relationships between pairs. For example if ETH dips, BTC has a certain probability to respond to that in a certain way.